<a href="https://colab.research.google.com/github/diogocascimiro/ALMG/blob/main/notebooks/primeiro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# [TITULOS DE INTERESSE -> CSV 2 COLUNAS]
# - intervalo sobreposto: pag_fim = próxima página onde começa um novo título (sem "- 1" no texto; aqui vira -1 no cálculo)
# - ignora títulos "pai" quando houver um "filho" na mesma página (ex.: "OFÍCIOS" vs "OFÍCIOS - ...")

import re
import csv
import os
from pathlib import Path

# ---- 0) Dependências (pypdf) ----
try:
    from pypdf import PdfReader
except Exception:
    !pip -q install pypdf
    from pypdf import PdfReader

# ---- 1) Config ----
PADROES_TITULOS = [
    r"^VOTAÇÕES\s+NOMINAIS\b",
    r"^OF[ÍI]CIOS\b",
    r"^REQUERIMENTOS\b",
    r"^ERRATAS\b",
]

# Títulos "pai" que devem ser ignorados quando existir um "filho" na mesma página
# Ex.: se houver "OFÍCIOS - PEDIDO DE INFORMAÇÃO" na página, não registrar "OFÍCIOS" nessa página.
TITULOS_PAI = [
    r"^OF[ÍI]CIOS\b$",
    r"^REQUERIMENTOS\b$",
    r"^ERRATAS\b$",
]

# Regex para identificar marcador de página (texto extraído costuma ter "Página 12")
RE_PAG = re.compile(r"\bP[ÁA]GINA\s+(\d{1,4})\b", re.IGNORECASE)

# ---- 2) Escolha do PDF (upload ou caminho) ----
print("1) Se você já tem o PDF no Drive, use um caminho tipo /content/drive/MyDrive/....")
print("2) Se preferir, faça upload agora.\n")

pdf_path = None

# Tenta montar Drive (não falha se você não quiser)
try:
    from google.colab import drive, files
    _COLAB = True
except Exception:
    _COLAB = False

if _COLAB:
    modo = input("Digite 'U' para upload ou 'C' para informar caminho: ").strip().upper()
    if modo == "U":
        up = files.upload()
        if not up:
            raise SystemExit("Nenhum arquivo enviado.")
        pdf_path = next(iter(up.keys()))
        print(f"Upload OK: {pdf_path}")
    else:
        pdf_path = input("Cole o caminho completo do PDF: ").strip()
else:
    pdf_path = input("Cole o caminho completo do PDF: ").strip()

pdf_path = str(pdf_path)

if not os.path.exists(pdf_path):
    raise FileNotFoundError(f"PDF não encontrado: {pdf_path}")

# ---- 3) Extrai títulos por página ----
padroes_comp = [re.compile(p, re.IGNORECASE) for p in PADROES_TITULOS]
pais_comp = [re.compile(p, re.IGNORECASE) for p in TITULOS_PAI]

def limpa_linha(s: str) -> str:
    s = s.replace("\u00a0", " ")
    s = re.sub(r"[ \t]+", " ", s).strip()
    return s

def eh_titulo_interesse(linha: str) -> bool:
    return any(p.match(linha) for p in padroes_comp)

def eh_pai(linha: str) -> bool:
    return any(p.match(linha) for p in pais_comp)

def eh_filho_do_pai(linha: str, pai: str) -> bool:
    # filho = pai seguido de " - algo"
    # ex.: OFÍCIOS - PEDIDO DE INFORMAÇÃO
    pai_norm = pai.upper()
    ln = linha.upper()
    return ln.startswith(pai_norm + " -")

reader = PdfReader(pdf_path)

# lista de ocorrências: (pagina_num, titulo_original)
ocorrencias = []

for i, page in enumerate(reader.pages):
    texto = page.extract_text() or ""
    linhas = [limpa_linha(x) for x in texto.splitlines() if limpa_linha(x)]

    # tenta detectar "Página N" nessa página (se não achar, usa i+1 como fallback)
    pag_match = None
    for ln in linhas[:80]:  # procura no começo
        m = RE_PAG.search(ln)
        if m:
            pag_match = int(m.group(1))
            break
    pag_num = pag_match if pag_match is not None else (i + 1)

    # coleta títulos candidatos
    candidatos = [ln for ln in linhas if eh_titulo_interesse(ln)]

    if not candidatos:
        continue

    # regra: se tiver filho, ignora pai na mesma página
    # exemplo: se existir "OFÍCIOS - ..." remove "OFÍCIOS"
    filtrados = []
    for c in candidatos:
        if eh_pai(c):
            # existe algum filho desse pai nessa mesma página?
            if any(eh_filho_do_pai(x, c) for x in candidatos):
                continue
        filtrados.append(c)

    # salva
    for t in filtrados:
        ocorrencias.append((pag_num, t))

if not ocorrencias:
    raise SystemExit("Nenhum título encontrado com os padrões atuais. Ajuste PADROES_TITULOS.")

# ordena por página e por ordem de aparição (já está, mas garante)
ocorrencias.sort(key=lambda x: x[0])

# ---- 4) Monta intervalos sobrepostos (pag_ini até prox_pag_ini-1) ----
# Se houver múltiplos títulos na mesma página, cada um vira um intervalo "pag-pag"
# (é o padrão mais seguro; se você quiser outra lógica, ajusta fácil)
itens = []
for idx, (pag_ini, titulo) in enumerate(ocorrencias):
    # encontra próxima ocorrência com página diferente
    prox_pag = None
    for j in range(idx + 1, len(ocorrencias)):
        if ocorrencias[j][0] != pag_ini:
            prox_pag = ocorrencias[j][0]
            break

    if prox_pag is None:
        # último título: vai até o fim do PDF (pelo número de páginas do arquivo)
        pag_fim = len(reader.pages)
    else:
        pag_fim = prox_pag - 1

    if pag_fim < pag_ini:
        pag_fim = pag_ini

    intervalo = f"{pag_ini}-{pag_fim}" if pag_ini != pag_fim else f"{pag_ini}"
    itens.append((intervalo, titulo))

# ---- 5) Salva CSV (tab) ----
out_csv = Path("titulos_interesse_2col.csv")
with open(out_csv, "w", newline="", encoding="utf-8") as f:
    w = csv.writer(f, delimiter="\t")
    w.writerow(["intervalo", "titulo_original"])
    for intervalo, titulo in itens:
        w.writerow([intervalo, titulo])

print(f"\nOK: {out_csv.resolve()}")
print("\nPrévia:")
for row in itens[:15]:
    print(f"{row[0]}\t{row[1]}")

# ---- 6) (Colab) download fácil ----
if _COLAB:
    try:
        from google.colab import files
        files.download(str(out_csv))
    except Exception:
        pass


1) Se você já tem o PDF no Drive, use um caminho tipo /content/drive/MyDrive/....
2) Se preferir, faça upload agora.

Digite 'U' para upload ou 'C' para informar caminho: U


Saving L20251219.pdf to L20251219 (1).pdf
Upload OK: L20251219 (1).pdf

OK: /content/titulos_interesse_2col.csv

Prévia:
81-84	OFÍCIOS
85-149	REQUERIMENTOS
150-151	requerimentos incidentais referentes à Proposta de Emenda à Constituição nº 24/2023 sejam votados nominalmente. Submetidos a
152-186	Requerimentos nºs 15.735 a 15.739/2025. Submetido a discussão e votação, é aprovado o Parecer de Redação Final do Projeto de Lei
187	OFÍCIOS
188-189	REQUERIMENTOS APROV ADOS
188-189	REQUERIMENTOS APROV ADOS
190	ERRATAS


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>